In [1]:
import ffisearch_B as ff
import numpy as np
import os
import common as cmn
from astropy.table import QTable, vstack
import lightkurve as lk
import re
import pandas as pd

In [2]:
ticid = 165791889
sigma_upper = 4.
sigma_lower = 12.
window_length = 0.8
method = 'biweight'
num_threads = 1

target_list = pd.read_csv("targets.csv")

tic_index = np.where(target_list['ID'] == ticid)[0][0]

mass = target_list['mass'][tic_index]
radius = target_list['rad'][tic_index]  

In [13]:
light_curve_direc = cmn.light_curve_direc
save_direc = cmn.transit_search_direc

In [4]:
dir_list = np.asarray(os.listdir(light_curve_direc))
#get file names for the given TIC ID
mask = np.asarray([(str(ticid).zfill(12) in i) for i in dir_list])
masked_dir_list = dir_list[mask]
#Open the light curve files and 
light_curves2 = []
sectors = []
full_2d_list = []
for i in masked_dir_list:
    #Open the light curve using astropy
    read_lc = QTable.read((light_curve_direc + i), format='fits', astropy_native=True)
    #Correct the time column to TESS times (BTJD)
    read_lc['time'] = lk.LightCurve(read_lc).time.btjd
    light_curves2.append(read_lc)
    #Determine the sector using the light curve file name
    sector = re.findall('sector_00(.*)_tic', i)[0]
    sectors.append(int(sector))
    full_2d_list.append([read_lc, int(sector)])

In [5]:
flux_id = ff.determine_best_flux(light_curves2)

In [6]:
sorted_sectors = sorted(sectors)
consecutive_sectors = []
sec_group = [sorted_sectors[0]]
for i in range(1, len(sorted_sectors)):
    if sorted_sectors[i] == (sorted_sectors[i-1] + 1):
        sec_group.append(sorted_sectors[i])
    else:
        consecutive_sectors.append(sec_group)
        sec_group = [sorted_sectors[i]]
consecutive_sectors.append(sec_group)
light_curves = []
for i in consecutive_sectors:
    #stack the light curves using astropy vstack
    stack = vstack([full_2d_list[j][0] for j in range(len(full_2d_list)) if full_2d_list[j][1] in i])
    #Resort the new light curve so that it is in sequential order
    stack.sort('time')
    light_curves.append(stack)

In [7]:
all_flat = []
all_times = []
for i in range(len(light_curves)):
    time = light_curves[i]['time'].value
    flux = light_curves[i][flux_id].value
    times, flat, trend = ff.flatten_lightcurve(time, flux, sigma_upper, sigma_lower, window_length, method)
    all_flat.append(flat)
    all_times.append(times)

In [8]:
all_results = []
for i in range(len(light_curves)):
    results = ff.search_for_transit(all_times[i], all_flat[i], mass=mass, radius=radius, num_threads=num_threads)
    all_results.append(results)

In [14]:
if 0.2*len(all_results) > 2:
    sec_thresh1 = int(0.2*len(all_results))
else:
    sec_thresh1 = 2

flag, sec_num = ff.flagging_criteria(all_results, sde_thresh = 7, sec_thresh=sec_thresh1, save_direc=save_direc)

In [15]:
print(flag)

False
